In [3]:
import torch
import torchvision
import numpy as np
from torchvision import datasets
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
from torch import optim
import matplotlib.pyplot as plt
from torch.utils.data.sampler import SubsetRandomSampler
import math
import UNet
import os
import random
from PIL import Image

In [ ]:
mean = [0.429, 0.431, 0.397]
std  = [1, 1, 1]
data_transform = transforms.Compose([    
        transforms.Resize((224,224)),    
        transforms.ToTensor(),
        transforms.Normalize(mean,std)
    ])
data = dataset_loader(r'Data set\train',transforms=data_transform)
data.frames

In [ ]:
class dataset_loader():
    def __init__(self,dataset_path,transforms = None):
        self.data_path = dataset_path
        self.transforms = transforms
        if not os.path.exists(dataset_path):
            raise(RuntimeError("path doesn't exist !"))
        self.frames = self.get_paths()
    def get_paths(self):
        images = []
        dirs = os.listdir(self.data_path)
        for folder in dirs:
            folder_path = os.path.join(self.data_path,folder)
            if not os.path.isdir(folder_path):
                continue
            folder_images = []
            for img in os.listdir(folder_path):
                folder_images.append(os.path.join(folder_path,img))
            images.append(folder_images)
        return images
    def __getitem__(self,index):
        frame_0_indx = random.randint(0,len(self.frames[index]) - 9) #9 = 1(index base = 0 not 1) + 8(7 frames between I0,I1)
        frame_1_indx = frame_0_indx + 8
        frame_t_indx = random.randint(frame_0_indx + 1 ,frame_1_indx - 1)
        image_0 = Image.open(self.frames[index][frame_0_indx])
        image_t = Image.open(self.frames[index][frame_t_indx])
        image_1 = Image.open(self.frames[index][frame_1_indx])
        if self.transforms is not None:
            image_0 = self.transforms(image_0)
            image_t = self.transforms(image_t)
            image_1 = self.transforms(image_1)
        return (image_0,image_t,image_1),frame_t_indx - frame_0_indx - 1
    def __len__(self):
        return len(self.frames)

In [11]:
x,y = np.meshgrid(np.linspace(-1,1,5),np.linspace(-1,1,3))
print(x)
print("")
print(y.astype(Double))

[[-1.  -0.5  0.   0.5  1. ]
 [-1.  -0.5  0.   0.5  1. ]
 [-1.  -0.5  0.   0.5  1. ]]



NameError: name 'Double' is not defined

In [33]:
from torch.autograd import Variable
def apply_flow(img,flow,device):
    """
        warp an image according to the optical flow
        img : [B, C, H, W] ex: torch.Size([3, 3, 352, 352])
        flow: [B, 2, H, W] ex: torch.Size([3, 2, 352, 352])
        
        F.grid_sample() : for input with shape (B, C, Hin, Win)
        and grid with shape (B, Hout, Wout, 2), 
        the output will have shape (B, C, Hout, Wout).
    """
    B,C,H,W = img.size()
    gridx,gridy = np.meshgrid(np.linspace(-1,1,H),np.linspace(-1,1,W))
    gridx = np.reshape(gridx,(1,1,H,W))
    gridy = np.reshape(gridy,(1,1,H,W))
    gridx = np.repeat(gridx,repeats=B,axis=0)
    gridy = np.repeat(gridy,repeats=B,axis=0)
    grid = np.concatenate((gridx,gridy),axis=1)
    grid = torch.from_numpy(grid).float().to(device)
    
    normalized_flow =  flow + grid
    normalized_flow = normalized_flow.permute(0,2,3,1)#view(B,H,W,2)
    #print("normalized_flow 1 : {}".format(normalized_flow))
    new_img = F.grid_sample(img,normalized_flow,mode='bilinear')
    return new_img
def warp(x, flo):
        """
        warp an image/tensor (im2) back to im1, according to the optical flow
        x: [B, C, H, W] (im2)
        flo: [B, 2, H, W] flow
        """
        B, C, H, W = x.size()
        # mesh grid 
        xx = torch.arange(0, W).view(1,-1).repeat(H,1)
        yy = torch.arange(0, H).view(-1,1).repeat(1,W)
        xx = xx.view(1,1,H,W).repeat(B,1,1,1)
        yy = yy.view(1,1,H,W).repeat(B,1,1,1)
        grid = torch.cat((xx,yy),1).float()
        if x.is_cuda:
            grid = grid.cuda()
        vgrid = Variable(grid) + flo
        
        # scale grid to [-1,1] 
        vgrid[:,0,:,:] = 2.0*vgrid[:,0,:,:]/max(W-1,1)-1.0
        vgrid[:,1,:,:] = 2.0*vgrid[:,1,:,:]/max(H-1,1)-1.0
        vgrid = vgrid.permute(0,2,3,1)     
        print("vgrid 1 : {}".format(vgrid))
        output = nn.functional.grid_sample(x, vgrid)
        mask = torch.autograd.Variable(torch.ones(x.size())).cuda()
        mask = nn.functional.grid_sample(mask, vgrid)
        mask[mask<0.9999] = 0
        mask[mask>0] = 1
        
        return output

In [34]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
I0_test = np.zeros(shape=(2,3,4,4))
flow_t = np.zeros(shape=(2,2,4,4))
I0_test = torch.from_numpy(I0_test).float().to(device)
flow_t = torch.from_numpy(flow_t).float().to(device)
g_I0_F_t_0_2 = apply_flow(I0_test, flow_t, device)
g_I0_F_t_0_2 = warp(I0_test, flow_t)

normalized_flow 1 : tensor([[[[-1.0000, -1.0000],
          [-0.3333, -1.0000],
          [ 0.3333, -1.0000],
          [ 1.0000, -1.0000]],

         [[-1.0000, -0.3333],
          [-0.3333, -0.3333],
          [ 0.3333, -0.3333],
          [ 1.0000, -0.3333]],

         [[-1.0000,  0.3333],
          [-0.3333,  0.3333],
          [ 0.3333,  0.3333],
          [ 1.0000,  0.3333]],

         [[-1.0000,  1.0000],
          [-0.3333,  1.0000],
          [ 0.3333,  1.0000],
          [ 1.0000,  1.0000]]],


        [[[-1.0000, -1.0000],
          [-0.3333, -1.0000],
          [ 0.3333, -1.0000],
          [ 1.0000, -1.0000]],

         [[-1.0000, -0.3333],
          [-0.3333, -0.3333],
          [ 0.3333, -0.3333],
          [ 1.0000, -0.3333]],

         [[-1.0000,  0.3333],
          [-0.3333,  0.3333],
          [ 0.3333,  0.3333],
          [ 1.0000,  0.3333]],

         [[-1.0000,  1.0000],
          [-0.3333,  1.0000],
          [ 0.3333,  1.0000],
          [ 1.0000,  1.0000]]]], d